In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [6]:
df = pd.read_csv('dataset (3).csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


## Model 1 - Baseline Linear Model

In [7]:
# Set base price and alpha
BASE_PRICE = 10.0
ALPHA = 5.0

# Calculate occupancy ratio
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']

# Model 1 - Baseline price update
df['PriceModel1'] = BASE_PRICE + ALPHA * df['OccupancyRatio']

## Model 2 - Demand Based function Prediction

In [15]:
# Encode vehicle type weight
vehicle_weights = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5,
    'cycle': 0.0  # negligible emissions
}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)

# Demand function parameters
alpha, beta, gamma, delta, epsilon = 0.4, 0.3, 0.2, 0.2, 0.3
lambda_ = 0.5

# Encode traffic levels numerically
traffic_encoding = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_encoding)

# Computed raw demand
df['DemandRaw'] = (
    alpha * df['OccupancyRatio'] +
    beta * df['QueueLength'] -
    gamma * df['TrafficLevel'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['VehicleWeight']
)

# Normalized demand
df['DemandNorm'] = (df['DemandRaw'] - df['DemandRaw'].min()) / (df['DemandRaw'].max() - df['DemandRaw'].min())

# dynamic price (bounded between 0.5x and 2x of base)
df['PriceModel2'] = BASE_PRICE * (1 + lambda_ * df['DemandNorm'])
df['PriceModel2'] = df['PriceModel2'].clip(lower=BASE_PRICE * 0.5, upper=BASE_PRICE * 2.0)

# for clarity
df['PriceModel2'] = df['PriceModel2'].round(2)



In [17]:
df.drop(columns=['traffic_level'], inplace=True)
df


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,OccupancyRatio,PriceModel1,VehicleWeight,TrafficLevel,DemandRaw,DemandNorm,PriceModel2
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,0.105719,10.528596,1.0,1,0.442288,0.101014,10.51
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,0.110919,10.554593,1.0,1,0.444367,0.101447,10.51
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,0.138648,10.693241,1.0,1,0.755459,0.166285,10.83
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,0.185442,10.927210,1.0,1,0.774177,0.170187,10.85
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,0.259965,11.299827,0.5,1,0.653986,0.145136,10.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00,0.790104,13.950521,1.5,2,2.166042,0.460281,12.30
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00,0.774479,13.872396,1.0,1,1.309792,0.281820,11.41
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00,0.745833,13.729167,0.0,1,0.998333,0.216906,11.08
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00,0.688021,13.440104,1.0,1,0.975208,0.212086,11.06


In [52]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [56]:
import pathway as pw

class ParkingStream(pw.Schema):
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    OccupancyRatio: float
    PriceModel1: float
    VehicleWeight: float
    TrafficLevel: int
    DemandRaw: float
    DemandNorm: float
    PriceModel2: float
    Timestamp: str


In [53]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,OccupancyRatio,PriceModel1,VehicleWeight,TrafficLevel,DemandRaw,DemandNorm,PriceModel2,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,0.105719,10.528596,1.0,1,0.442288,0.101014,10.51,2016-10-04 07:59:00
1,7872,Broad Street,690,26.137958,91.740994,178,car,low,2,0,04-10-2016,07:59:00,0.257971,11.289855,1.0,1,0.803188,0.176233,10.88,2016-10-04 07:59:00
2,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,0.197500,10.987500,0.5,1,0.629000,0.139929,10.70,2016-10-04 07:59:00
3,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,0.384279,11.921397,1.0,1,0.853712,0.186763,10.93,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,0.319792,11.598958,0.0,1,0.527917,0.118861,10.59,2016-10-04 07:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,1.000000,15.000000,1.0,1,1.100000,0.238095,11.19,2016-12-19 16:30:00
18364,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,0.334489,11.672444,0.0,1,0.533795,0.120086,10.60,2016-12-19 16:30:00
18365,10495,Others-CCCPS105a,2009,26.147473,91.728049,1345,bike,low,2,0,19-12-2016,16:30:00,0.669487,13.347437,0.5,1,0.817795,0.179278,10.90,2016-12-19 16:30:00
18366,7871,BHMNCPNST01,485,26.140048,91.730972,257,car,low,2,0,19-12-2016,16:30:00,0.529897,12.649485,1.0,1,0.911959,0.198903,10.99,2016-12-19 16:30:00


In [76]:
df.to_csv("parking_stream.csv", index=False)

parking_input = pw.demo.replay_csv(
     "parking_stream.csv",
    schema=ParkingStream,
    input_rate=1000,
)


# Select relevant fields to stream/process
output_table = parking_input.select(
    Code=pw.this.SystemCodeNumber,
    Price1=pw.this.PriceModel1,
    Price2=pw.this.PriceModel2,
    Lat=pw.this.Latitude,
    Lng=pw.this.Longitude,
    UpdatedAt=pw.this.Timestamp,

)

# Output to CSV (for testing) or stream to another sink
pw.io.csv.write(output_table, "live_price_output.csv")


In [58]:
pip install bokeh pandas watchdog


In [77]:
import panel as pn
import bokeh.plotting
from bokeh.models import HoverTool
pn.extension()

from bokeh.io import output_notebook, show
output_notebook()

In [78]:
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Live Parking Prices (Pathway Stream)",
        x_axis_type="datetime",
        x_axis_label="Time",
        y_axis_label="Price (₹)",
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    fig.line("UpdatedAt", "Price1", source=source, color="green", legend_label="PriceModel1", line_width=2)
    fig.circle("UpdatedAt", "Price1", source=source, size=5, color="green")

    fig.line("UpdatedAt", "Price2", source=source, color="blue", legend_label="PriceModel2", line_width=2)
    fig.circle("UpdatedAt", "Price2", source=source, size=5, color="blue")

    fig.legend.location = "top_left"
    fig.add_tools(HoverTool(
        tooltips=[("Time", "@UpdatedAt{%F %T}"), ("Price1", "@Price1"), ("Price2", "@Price2")],
        formatters={"@UpdatedAt": "datetime"},
        mode='vline'
    ))

    fig.scatter("UpdatedAt", "Price1", source=source, size=5, color="green")
    fig.scatter("UpdatedAt", "Price2", source=source, size=5, color="blue")

    return fig


In [ ]:
viz = output_table.plot(price_plotter, sorting_col="UpdatedAt")
viz.servable()
pw.run()



Output()

In [82]:
# Bind the data statically to plot
df_plot = df[['Timestamp', 'PriceModel1', 'PriceModel2']].copy()

from bokeh.models import ColumnDataSource
source = ColumnDataSource(df_plot.rename(columns={"Timestamp": "UpdatedAt", "PriceModel1": "Price1", "PriceModel2": "Price2"}))
fig = price_plotter(source)

show(fig)
